In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import convolutionalNetwork
import dataset

In [ ]:
from convolutionalNetwork import ConvNet
network = ConvNet(32, 3)
model = network.initModel(32, 3)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 125, 125, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 32)        9

In [ ]:
ds = dataset.dataset(256, 256)
train_images = ds['Xtrain']
train_labels = ds['Ytrain']

In [ ]:
print((train_images.shape))

(2000, 256, 256, 3)


In [ ]:
print(train_labels[:2000].shape)

(2000, 10)


In [ ]:
train_images = train_images/255.0

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(train_images, train_labels[:2000],
          batch_size=100,
          epochs=5,
          validation_split=0.1,
          verbose=1)

Epoch 1/5
18/18 [==============================] - 595s 30s/step - loss: 1.9029 - accuracy: 0.3703 - val_loss: 2.4040 - val_accuracy: 0.0000e+00
Epoch 2/5
18/18 [==============================] - 180s 9s/step - loss: 0.8198 - accuracy: 0.5449 - val_loss: 1.5553 - val_accuracy: 0.0000e+00
Epoch 3/5
18/18 [==============================] - 139s 8s/step - loss: 0.7293 - accuracy: 0.5417 - val_loss: 0.8375 - val_accuracy: 0.0700
Epoch 4/5
18/18 [==============================] - 180s 10s/step - loss: 0.7074 - accuracy: 0.5511 - val_loss: 0.8122 - val_accuracy: 0.1000
Epoch 5/5
18/18 [==============================] - 252s 13s/step - loss: 0.7073 - accuracy: 0.5347 - val_loss: 0.6629 - val_accuracy: 0.7250


In [ ]:
print(len(train_images))

12


In [ ]:
# print(len(train_images))
# model.summary()
print(train_images)

[[[[ 73  87  86]
   [ 72  85  87]
   [ 77  89  93]
   ...
   [ 81  77  77]
   [ 79  76  78]
   [ 87  84  93]]

  [[ 79  91  93]
   [ 71  84  86]
   [ 75  87  91]
   ...
   [ 80  78  78]
   [ 79  77  79]
   [ 85  84  93]]

  [[ 76  86  91]
   [ 74  84  90]
   [ 94 106 110]
   ...
   [ 97  95  94]
   [ 85  86  87]
   [ 83  85  92]]

  ...

  [[ 31  42  40]
   [ 28  34  31]
   [ 20  31  28]
   ...
   [ 38  45  41]
   [ 23  28  29]
   [ 19  27  22]]

  [[ 33  44  42]
   [ 36  41  39]
   [ 43  55  50]
   ...
   [ 28  33  32]
   [ 23  28  32]
   [ 25  32  29]]

  [[ 25  35  36]
   [ 39  45  43]
   [ 48  60  54]
   ...
   [ 17  21  22]
   [ 17  21  26]
   [ 19  25  25]]]


 [[[131 127 117]
   [125 124 114]
   [154 150 136]
   ...
   [189 178 176]
   [173 165 164]
   [179 178 175]]

  [[ 94  95  86]
   [ 91  96  88]
   [103 104  94]
   ...
   [159 165 164]
   [122 137 135]
   [144 147 148]]

  [[112 103  93]
   [110 108  98]
   [135 134 121]
   ...
   [178 173 174]
   [154 162 162]
   [157 168